# Akoya Academy Webinar: PhenoImager Analysis
This notebook contains the necessary libraries and steps to import the single cell data from QuPath into Python. The files exported from QuPath are stored under `data/` folder.
## Step-1: Data Import from QuPath into Python
Step-1 covers the data import portion. The key stages are obtaining single cell level data, splitting the expression and cell metadata (such as x,y location of each cell, sample/region it belongs to, and any other information per sample is stored for each cell in an `AnnData` (Annotated Data` object, which allows for easy access by other Python/R libraries like Scanpy, Scimap, and Seurat (R.)))

### Import necessary libararies

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import anndata as ad
import pandas as pd
import scanpy as sc
import numpy as np

import seaborn as sns; sns.set(color_codes=True)

#ignore warnings
import warnings
warnings.filterwarnings('ignore')


### Read annotation and detection measurements 

The annotation measurements in QuPath contain the per sample information like number of regions, tissue area, etc.  For this example, they are stored in a single file called `data/annotation_measurements.tsv`. We do not use this information for this Webinar, but it is useful if we need to compute cell-type densities.

The detection measurements contain information for each cell measured. These can either be expored per sample or one file for all the samples. For this example, they are stored in a single file called `data/measurements.tsv`

In [2]:
# Read annotation measurements
annotDF = pd.read_csv('data/annotation_measurements.tsv', sep='\t')
annotDF.reset_index(drop=True,inplace=True)
annotDF.head()



,Image,Object ID,Name,Class,Parent,ROI,Centroid X µm,Centroid Y µm,Num Detections,Area µm^2,Perimeter µm
0,NSCLC_S2.ome.tif,d7123729-d749-461f-bae4-7db8beff215e,PathAnnotationObject,NaN,Image,Polygon,2965.9,6040.8,171391,31963110.1,32474.1
1,NSCLC_S1.ome..tif,05cb5fca-490b-4c7e-bd2f-67c0cf26d834,PathAnnotationObject,NaN,Image,Polygon,3613.8,3585.6,266898,30943724.1,22996.1


In [3]:
# Read detection measurements
detectDF = pd.read_csv('data/measurements.tsv', sep='\t')
detectDF.reset_index(drop=True,inplace=True)
detectDF.head()



,Image,Object ID,Name,Class,Parent,ROI,Centroid X µm,Centroid Y µm,Nucleus: Area µm^2,Nucleus: Length µm,...,Autofluorescence: Membrane: Mean,Autofluorescence: Membrane: Median,Autofluorescence: Membrane: Min,Autofluorescence: Membrane: Max,Autofluorescence: Membrane: Std.Dev.,Autofluorescence: Cell: Mean,Autofluorescence: Cell: Median,Autofluorescence: Cell: Min,Autofluorescence: Cell: Max,Autofluorescence: Cell: Std.Dev.
0,NSCLC_S2.ome.tif,625cfac1-f27c-4294-a3f5-e6600babaf86,PathCellObject,NaN,PathAnnotationObject,Polygon,1305.2,367.39,27.2261,21.5441,...,10.7301,10.6507,7.3216,16.6454,2.0182,10.5371,10.1653,7.1624,16.7358,1.7164
1,NSCLC_S2.ome.tif,35f49979-852f-470f-bb7f-2ae483d3201c,PathCellObject,NaN,PathAnnotationObject,Polygon,1320.5,367.98,32.7638,20.9805,...,6.7584,6.2670,3.3218,11.9521,2.3786,7.8955,8.2150,2.8802,13.1822,2.7342
2,NSCLC_S2.ome.tif,66375609-8d8c-4b11-9256-59fce3566543,PathCellObject,NaN,PathAnnotationObject,Polygon,1249.2,371.28,46.4503,26.3656,...,9.2917,9.0461,6.8184,14.1078,1.5470,9.3994,9.2874,4.8943,14.9434,1.6165
3,NSCLC_S2.ome.tif,37e3e1d7-266e-441f-93e8-76daf7e86305,PathCellObject,NaN,PathAnnotationObject,Polygon,1238.0,370.61,30.1428,22.1930,...,9.8174,9.5429,5.4999,15.0943,2.7274,9.4347,9.1056,5.1311,16.3295,2.3848
4,NSCLC_S2.ome.tif,4ff9a657-62ea-45fd-a654-b9f4b960db60,PathCellObject,NaN,PathAnnotationObject,Polygon,1329.6,374.55,17.5936,15.5740,...,7.2549,6.6194,3.2507,14.0007,2.9528,8.6809,8.6351,3.2507,17.3481,3.2435


In [4]:
# Display total number of cells across the two datasets
detectDF.shape

(438289, 181)

In order to extract the protein expression measurements, here we use the avg. intensity values per cell stored in a column that matches: `Cell: Mean`. For markers expressed in nucleus alone, it may be appropriate to get `Nucleus: Mean` measurement instead. 

In [5]:
# Extract the columns with the mean protein expression values
# This includes DAPI and Autofluorescence, which we will remove later
protDF = detectDF[[col for col in detectDF.columns if 'Mean' in col and 'Cell:' in col]]
protDF.head()

,DAPI: Cell: Mean,CD8 (Opal 480): Cell: Mean,CD4 (Opal 520): Cell: Mean,CD3E (Opal 570): Cell: Mean,CD20 (Opal 620): Cell: Mean,PanCK (Opal 690): Cell: Mean,CD68 (Opal 780): Cell: Mean,Autofluorescence: Cell: Mean
0,12.1437,0.0000,0.9597,0.4113,1.4724,0.1294,0.1504,10.5371
1,15.9427,0.0105,1.1708,0.4397,1.1722,0.1245,0.8000,7.8955
2,12.0581,0.0000,0.5207,0.1898,1.2616,0.0821,0.2085,9.3994
3,12.1289,0.0000,0.7398,0.2709,1.3197,0.1429,0.2085,9.4347
4,7.0277,0.0000,0.7252,0.3777,1.2804,0.1218,0.4423,8.6809


In [6]:
# clean-up column names for easy access
newNames = {col:col.split(':')[0].split(' ')[0] for col in protDF.columns}
protDF.rename(newNames, inplace=True, axis=1)
protDF.head()

,DAPI,CD8,CD4,CD3E,CD20,PanCK,CD68,Autofluorescence
0,12.1437,0.0000,0.9597,0.4113,1.4724,0.1294,0.1504,10.5371
1,15.9427,0.0105,1.1708,0.4397,1.1722,0.1245,0.8000,7.8955
2,12.0581,0.0000,0.5207,0.1898,1.2616,0.0821,0.2085,9.3994
3,12.1289,0.0000,0.7398,0.2709,1.3197,0.1429,0.2085,9.4347
4,7.0277,0.0000,0.7252,0.3777,1.2804,0.1218,0.4423,8.6809


In [7]:
# Remove DAPI and Autofluorescence
protDF = protDF.drop(['DAPI','Autofluorescence'], axis=1)
protDF

,CD8,CD4,CD3E,CD20,PanCK,CD68
0,0.0000,0.9597,0.4113,1.4724,0.1294,0.1504
1,0.0105,1.1708,0.4397,1.1722,0.1245,0.8000
2,0.0000,0.5207,0.1898,1.2616,0.0821,0.2085
3,0.0000,0.7398,0.2709,1.3197,0.1429,0.2085
4,0.0000,0.7252,0.3777,1.2804,0.1218,0.4423
...,...,...,...,...,...,...
438284,0.3839,11.9009,1.1177,0.1444,0.0336,6.7729
438285,2.7588,6.0693,2.5121,0.0706,0.0263,2.2799
438286,3.2464,12.4591,4.0111,1.1242,0.0430,0.6164
438287,0.0000,16.7197,11.1259,0.9914,0.0870,0.3221


For each cell we also obtain the centroid location (in µm) and also cell area (in µm^2) and sample ID. It may be appropriate to also store information on responders and non-responders at this stage and the region ID (ROI) for each sample.

In [8]:
spatialCols = ['Centroid X µm','Centroid Y µm','Cell: Area µm^2','Image']
spatialNames  = {'Centroid X µm':'spatial_X','Centroid Y µm': 'spatial_Y','Cell: Area µm^2': 'Area','Image':'ImageID'}
spatDF = detectDF[spatialCols]
spatDF.rename(columns=spatialNames, inplace=True)
# clean up ImageID column by removing the ome.tif file extension
spatDF['ImageID'] = spatDF['ImageID'].apply(lambda x: x.split('.')[0])
spatDF

,spatial_X,spatial_Y,Area,ImageID
0,1305.2,367.39,128.2195,NSCLC_S2
1,1320.5,367.98,131.9781,NSCLC_S2
2,1249.2,371.28,183.6812,NSCLC_S2
3,1238.0,370.61,112.4868,NSCLC_S2
4,1329.6,374.55,135.7821,NSCLC_S2
...,...,...,...,...
438284,3673.9,7003.10,21.4722,NSCLC_S1
438285,3611.9,7002.50,64.0917,NSCLC_S1
438286,3641.1,7003.30,22.8487,NSCLC_S1
438287,3707.5,7003.70,10.0509,NSCLC_S1


### `AnnData` object used for downstream analysis

The final `AnnData` object is created in this stage, by specifying the protein and spatial measurment components.

In [9]:
# Load data
# combine the data and metadata file to generate the AnnData object
adata = ad.AnnData (protDF)
adata.obs = spatDF

### Save `AnnData` object

We save this annotated data object for the next step, evaluating Sample Quality.

In [10]:
adata.write_h5ad('data/adata.h5ad')

... storing 'ImageID' as categorical


In [ ]:

def computeTop20Btm10(ad):
    # return a dataframe with sampleID, Protein, ratio of top20/btm10
    top20btm10DF = pd.DataFrame(columns=['ImageID','Protein','top20btm10'])

    # for each sample
    for sID in ad.obs.ImageID.unique():
        subAD = ad[ad.obs.ImageID == sID]
        for x in subAD.var_names:
            aX = subAD[:,x].X.flatten()
            # compute 20 largest values in aX
            top20 = np.sort(aX)[-20:]
            # compute the mean of bottom 10th percentile of aX
            btm10 = np.sort(aX)[:int(len(aX)*0.1)]
            print(aX.shape, top20.shape, btm10.shape)
            top20btm10 = np.mean(top20)/np.mean(btm10)
            top20btm10DF = top20btm10DF.append({'ImageID':sID,'Protein':x,'top20btm10':top20btm10}, ignore_index=True)
        

In [ ]:
pd.set_option('display.float_format', lambda x: '%.4f' % x)
computeTop20Btm10(adata)